# 00.한국어전처리패키지


- 한국어 전처리 패키지 : https://wikidocs.net/92961
- soyNLP : https://github.com/lovit/soynlp

# 새로운 텍스트전처리 과정

## 01.데이터 로드

In [1]:
!pip install konlpy
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 44.1 MB/s 
--2022-08-04 15:01:25--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-08-04 15:01:25--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8515c9155eeef0a57506513a5e.dl.dropboxusercontent.com/cd/0/get/BqaQPLKXXeKdn4SeY1Vha_mabClIOwVT34kMNDECR7wP3KbOZXfatOJ_LWjnykCMnGt6rnFmPYXuDW-u7pcJUosOzy8YmvH4NRSEM

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd 

path = '/content/drive/MyDrive/NLP미니플젝-10조/Data/추가 데이터/'

train_df = pd.read_csv(path+"violence_train_all.csv")
test_df = pd.read_csv(path+"violence_valid_all.csv")

## 02.결측치 확인

In [4]:
print(train_df['문장'].isna().any())
print(train_df['clean'].isna().any())
print(train_df['violence'].isna().any())
print(test_df['문장'].isna().any())
print(test_df['clean'].isna().any())
print(test_df['violence'].isna().any())

False
False
False
False
False
False


## 03.결측치 행 제거

In [5]:
train_df = train_df.dropna(axis=0)
test_df = test_df.dropna(axis=0)

## 04.clean + violence 합이 1 아니면 제거 

In [6]:
# 탐색
remove_i = train_df[train_df['clean']+train_df['violence'] != 1.0].index 
print(remove_i)

# 제거
train_df.drop(remove_i,axis='index',inplace=True)

Int64Index([], dtype='int64')


In [7]:
# 탐색
remove_i = test_df[test_df['clean']+test_df['violence'] != 1.0].index 
print(remove_i)

# 제거
test_df.drop(remove_i,axis='index',inplace=True)

Int64Index([], dtype='int64')


## 05.텍스트전처리

- 한국어 전처리 패키지 : https://wikidocs.net/92961
- soyNLP : https://github.com/lovit/soynlp

### 한글과 띄어쓰기를 제외한 모든 부분을 제거

In [8]:
import re

In [9]:
a_list = []
for i in range(len(train_df)):
  text = train_df.iloc[i]['문장']
  hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 띄어쓰기를 제외한 모든 글자
  result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거한
  a_list.append(result)

train_df['문장'] = a_list

In [10]:
b_list = []
for i in range(len(test_df)):
  text = test_df.iloc[i]['문장']
  hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 띄어쓰기를 제외한 모든 글자
  result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거한
  b_list.append(result)
  
test_df['문장'] = b_list

### _#@#(개인정보) 포함한 행 제거

In [11]:
index_train = train_df[train_df['문장'].str.find("#@#") != -1].index
index_test = test_df[test_df['문장'].str.find("#@#") != -1].index

train_df = train_df.drop(index = index_train)
test_df = test_df.drop(index = index_test)

### 중복된 문자열 줄이기

soyNLP를 활용한 중복된 문자열 줄이기 : 잘 안됨  
기본 코드로 중복된 문자열 줄이기 : O ( 2자리 )

In [12]:
%%time
# 약 20초

def text_normalize(text):
    count = {}
    normalized_text = ""
    for c in text:
        if c not in count:
            count = {c: 1}
        else:
            count[c] += 1
        if count[c] <= 2:
            normalized_text += c

    return normalized_text

text_nor_list_train = []
text_nor_list_test = []

for i in range(len(train_df)):
  text = train_df.iloc[i]['문장']
  text_nor_list_train.append(text_normalize(text))

for i in range(len(test_df)):
  text = test_df.iloc[i]['문장']
  text_nor_list_test.append(text_normalize(text))

train_df['문장'] = text_nor_list_train
test_df['문장'] = text_nor_list_test

CPU times: user 44.5 s, sys: 189 ms, total: 44.7 s
Wall time: 44.8 s


## 06.형태소분석기

### kiwi

In [13]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 3.2 MB/s 
     |████████████████████████████████| 30.0 MB 1.2 MB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.13.0-py3-none-any.whl size=30103568 sha256=1e746cb470b793c55b32182fbb6e9993a54d3432b000fd44ebc7dfb09061eea9
  Stored in directory: /root/.cache/pip/wheels/ef/76/54/24f8a4f8de9818e0847b719cd7be1611f14eb1ce8d3dfe2460
Successfully built kiwipiepy-model


In [14]:
from kiwipiepy import Kiwi
kiwi = Kiwi()
kiwi.prepare()
for result, score in kiwi.analyze("형태소 분석 결과입니다", top_n=5):
    print(score, result, sep='\t')

-29.267728805541992	[Token(form='형태소', tag='NNG', start=0, len=3), Token(form='분석', tag='NNG', start=4, len=2), Token(form='결과', tag='NNG', start=7, len=2), Token(form='이', tag='VCP', start=9, len=1), Token(form='ᆸ니다', tag='EF', start=9, len=3)]
-50.80938720703125	[Token(form='형태소', tag='NNG', start=0, len=3), Token(form='분석', tag='NNG', start=4, len=2), Token(form='결과', tag='NNG', start=7, len=2), Token(form='입', tag='VV-R', start=9, len=1), Token(form='니다', tag='EF', start=10, len=2)]
-51.82143020629883	[Token(form='형태소', tag='NNG', start=0, len=3), Token(form='분석', tag='NNG', start=4, len=2), Token(form='결과', tag='NNG', start=7, len=2), Token(form='입', tag='VV-R', start=9, len=1), Token(form='니', tag='EC', start=10, len=1), Token(form='다', tag='MAG', start=11, len=1)]
-53.27513122558594	[Token(form='형태소', tag='NNG', start=0, len=3), Token(form='분석', tag='NNG', start=4, len=2), Token(form='결과', tag='NNG', start=7, len=2), Token(form='입', tag='VV-R', start=9, len=1), Token(form='니다', 

In [15]:
%%time

def tagging2(input_df,tag, parts):
  words_arr = []
  for i in input_df['문장']:
    text = i.strip() # 양쪽 공백제거
    #print(text)

    sentence = [] # 문장 : 단어를 문장으로 재조합하기 위해 ㅏㅅ용

    if len(text) == 0 or text == " " or text == "":
      words_arr.append("")
    else:
      result = tag.analyze(text)
      for i in result[0][0]:
        if i == ' ' or i == 'EOS' : continue
        word_temp = i[0] # 단어
        part = i[1] # 품사
        if not (part in parts): continue
        sentence.append(word_temp)
      words_arr.append(' '.join(sentence))
    
  return words_arr

# 44초 (train 기준)
train_df['kiwi'] = tagging2(train_df, kiwi,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EC','VV+EC'])
test_df['kiwi'] = tagging2(test_df, kiwi,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EC','VV+EC'])

CPU times: user 6min 55s, sys: 1.33 s, total: 6min 57s
Wall time: 7min 2s


### kahiii

In [16]:
!git clone https://github.com/kakao/khaiii.git

Cloning into 'khaiii'...
remote: Enumerating objects: 1024, done.
remote: Total 1024 (delta 0), reused 0 (delta 0), pack-reused 1024
Receiving objects: 100% (1024/1024), 33.03 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (438/438), done.


In [17]:
!pip install cmake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!mkdir build

In [19]:
!cd build && cmake /content/khaiii

-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test fma_compiles
-- Performing Test fma_compiles - Success
-- Performing Test fma_runs
-- Performing Test fma_runs - Success
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] 

In [20]:
!cd /content/build/ && make all

[  3%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Config.cpp.o
[  7%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Embed.cpp.o
[ 11%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/ErrPatch.cpp.o
[ 15%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/KhaiiiImpl.cpp.o
[ 19%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Morph.cpp.o
[ 23%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Preanal.cpp.o
[ 26%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Resource.cpp.o
[ 30%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Restore.cpp.o
[ 34%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Sentence.cpp.o
[ 38%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Tagger.cpp.o
[ 42%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Trie.cpp.o
[ 46%] Building CXX object CMakeFiles/obj_khai

In [21]:
!cd /content/build/ && make resource

INFO:root:config: {
    "cutoff": 1,
    "embed_dim": 35,
    "hidden_dim": 320,
    "model_id": "munjong.cut1.win4.sdo0.1.emb35.lr0.001.lrd0.9.bs500",
    "rsc_src": "../rsc/src",
    "window": 4
}
INFO:root:vocab.in: 5621 entries, 0 cutoff
INFO:root:vocab.out: 500 entries, 0 cutoff
INFO:root:restore.dic: 4303 entries
INFO:root:preanal.manual
INFO:root:preanal.auto
INFO:root:trie saved: /content/build/share/khaiii/preanal.tri
INFO:root:total nodes: 120812
INFO:root:expected size: 1932992
INFO:root:value saved: /content/build/share/khaiii/preanal.val
INFO:root:total entries: 83373
INFO:root:expected size: 593730
INFO:root:restore.key: 17212
INFO:root:restore.val: 68848
INFO:root:restore.one: 503
INFO:root:base.errpatch.manual
INFO:root:base.errpatch.auto
INFO:root:trie saved: /content/build/share/khaiii/errpatch.tri
INFO:root:total nodes: 2067
INFO:root:expected size: 33072
INFO:root:length saved: /content/build/share/khaiii/errpatch.len
INFO:root:expected size: 551
INFO:root:value sav

In [22]:
!cd /content/build && make install

Consolidate compiler generated dependencies of target obj_khaiii
[ 65%] Built target obj_khaiii
[ 69%] Built target khaiii
Consolidate compiler generated dependencies of target bin_khaiii
[ 76%] Built target bin_khaiii
Consolidate compiler generated dependencies of target test_khaiii
[100%] Built target test_khaiii
Install the project...
-- Install configuration: ""
-- Installing: /usr/local/include/khaiii
-- Installing: /usr/local/include/khaiii/KhaiiiApi.hpp
-- Installing: /usr/local/include/khaiii/khaiii_api.h
-- Installing: /usr/local/include/khaiii/khaiii_dev.h
-- Installing: /usr/local/share/khaiii
-- Installing: /usr/local/share/khaiii/restore.val
-- Installing: /usr/local/share/khaiii/errpatch.len
-- Installing: /usr/local/share/khaiii/hdn2tag.lin
-- Installing: /usr/local/share/khaiii/conv.2.fil
-- Installing: /usr/local/share/khaiii/errpatch.val
-- Installing: /usr/local/share/khaiii/preanal.tri
-- Installing: /usr/local/share/khaiii/restore.one
-- Installing: /usr/local/shar

In [23]:
!cd /content/build && make package_python

Run CPack packaging tool for source...
CPack: Create package using ZIP
CPack: Install projects
CPack: - Install directory: /content/khaiii
CPack: Create package
CPack: - package: /content/build/khaiii-0.4.zip generated.
Built target package_python


In [24]:
!pip install /content/build/package_python --use-feature=in-tree-build

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./build/package_python
  Created wheel for khaiii: filename=khaiii-0.4-py3-none-any.whl size=22883534 sha256=417bc9f99f37d299c75d7d7fc83a07ba99be8c92d2e3cda098cc9d460241c4ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-heooig11/wheels/79/cb/8c/aed91c3bafdd491bf3fcbed5809b53e50a508c6e167bbbeff8
Successfully built khaiii


In [25]:
from khaiii import KhaiiiApi
khaiii = KhaiiiApi()
for word in khaiii.analyze("안녕, 세상."):
    print(word)


안녕,	안녕/IC + ,/SP
세상.	세상/NNG + ./SF


In [26]:
# 형태소들을 넣을 빈 리스트
morphs_list = []

# 형태소단위로 튜플을 생성하여 한 리스트로 합치기
for word in khaiii.analyze("나는 학교에 간다"):
    for i in word.morphs:
      print(i.lex, i.tag)

나 NP
는 JX
학교 NNG
에 JKB
가 VV
ㄴ다 EC


In [27]:
%%time

def tagging3(input_df,tag, parts):
  words_arr = []
  for i in input_df['문장']:
    text = i.strip() # 양쪽 공백제거

    sentence = [] # 문장 : 단어를 문장으로 재조합하기 위해 ㅏㅅ용

    if len(text) == 0 or text == " " or text == "":
      words_arr.append("")
    else:
      result = tag.analyze(text)

      for word in result:
        for morphs in word.morphs:
          #print(morphs)
          if morphs == ' ' or morphs == 'EOS' or morphs == "": continue
          word_temp = morphs.lex # 단어
          part = morphs.tag # 품사
          if not (part in parts): continue
          sentence.append(word_temp)
      words_arr.append(' '.join(sentence))
  return words_arr

# 24초 (train 기준)
train_df['khaiii'] = tagging3(train_df, khaiii,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EC','VV+EC'])
test_df['khaiii'] = tagging3(test_df, khaiii,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EC','VV+EC'])

CPU times: user 5min 6s, sys: 961 ms, total: 5min 7s
Wall time: 5min 13s


### mecab

In [28]:
from konlpy.tag import Mecab

mecab = Mecab()

In [29]:
%%time

## "명사", "동사", "형용사"

def tagging(input_df,tag, parts):
  words_arr = []
  for i in input_df['문장']:
    text = i.strip() # 양쪽 공백제거
    #print(text)

    sentence = [] # 문장 : 단어를 문장으로 재조합하기 위해 ㅏㅅ용

    taggings = tag.pos(text)

    if len(text) == 0 or text == " " or text == "":
      words_arr.append("")
    else:
      for i in taggings:
        if i == 'EOS' or i == ' ' : 
          continue
        word_temp = i[0] # word 
        part = i[1] # 품사
        if not (part in parts): # ['Noun','Verb','Adjective']
          continue
        sentence.append(word_temp)
      words_arr.append(' '.join(sentence)) # 단어들을 문장으로 재조합 후, 리스트에 추가
    
  return words_arr

# 3초
# case 1 : 체언, 용언(동사,형용사)
# train_df['mecab'] = tagging(train_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA'])  
# test_df['mecab'] = tagging(test_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA'])  

# # case 2 : 체언, 용언, 관형사, 부사
# train_df['mecab'] = tagging(train_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ'])  
# test_df['mecab'] = tagging(test_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ'])  

# case 3 : 체언, 용언, 관형사, 부사, 조사
# 15만개 데이터 시간 : 
train_df['mecab'] = tagging(train_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EC','VV+EC'])  
test_df['mecab'] = tagging(test_df, mecab,['NNG','NNP','NNB','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EC','VV+EC']) 

CPU times: user 36.1 s, sys: 398 ms, total: 36.5 s
Wall time: 42.1 s


In [30]:
train_df

,문장,clean,violence,kiwi,khaiii,mecab
0,저 사람은 이미 다 인성 안좋은거 아는데도 에 나오네,0.0,1.0,저 사람 은 이미 다 인성 안 좋 거 알 는데 도 나오,저 사람 은 이미 다 인성 안 좋 거 알 는데 도 에 나 오 네,저 사람 은 이미 다 인성 안 좋 거 아 데 도 나오 네
1,예전에는 다 해줬으면서 아줌마가 되니까 몸도 무겁고 별수 없나봐,0.0,1.0,예전 에 는 다 하 어 주 으면서 아줌마 가 되 니까 몸 도 고 별 수 없 나 보,예전 에 는 다 하 여 주 으면서 아줌마 가 되 니까 몸 도 무겁 고 별수 없 나 봐,예전 에 는 다 해 으면서 아줌마 가 되 니까 몸 도 무겁 고 별수 없 나
2,공부 열심히 해서 보답해드려야지,1.0,0.0,공부 열심히 하 어서 보답 어 드리,공부 열심히 하 여서 보답 여 드 려야지,공부 열심히 해서 보답 지
3,요럴라고 민주화 운동했당께,0.0,1.0,요러 ᆯ라고 민주 운동,요 러 ㄹ 라고 민주 운동 당 께,요 민주 운동 당께
4,여경이란 소리 없던데 경찰이 맨몸 난동꾼 하나 제압 못해서 동료 기다리는 동안 편의...,0.0,1.0,여경 이 소리 없 경찰 이 몸 난동 하나 제압 못 어서 동료 기다리 동안 편의점 알...,여경 이 소리 없 던데 경찰 이 몸 난동 하나 제압 못 하 여서 동료 기다리 동안 ...,여경 이 소리 없 던데 경찰 이 맨몸 난동 하나 제압 못 해서 동료 기다리 동안 편...
...,...,...,...,...,...,...
302522,여사친이 뭐 했어,1.0,0.0,여사친 이 뭐 하,여사친 이 뭐 하 어,여 사친 이 어
302523,보지에 늘 새기겠노,0.0,1.0,보지 에 늘 새기,보지 에 늘 새기 노,보지 에 늘 새기 노
302524,대동강 돼지 농장 살처분 계획,1.0,0.0,대동강 돼지 농장 살처분 계획,대동강 돼지 농장 살처분 계획,대동강 돼지 농장 처분 계획
302525,저따구로 돈벌어서 일반인 만날 생각마,1.0,0.0,저따구 로 돈 벌 어서 일반인 만나 생각 말,저따구 로 돈 벌 어서 일반 이 만 나 생각마,저 따구 로 돈 벌 어서 일반인 각마


## 07.컬럼 정리

In [31]:
# 분할
train_mecab = train_df[['mecab','clean','violence']]
test_mecab = test_df[['mecab','clean','violence']]

# 각각 길이가 2미만 제거 
train_mecab = train_mecab[train_mecab['mecab'].str.len() >= 2]
test_mecab = test_mecab[test_mecab['mecab'].str.len() >= 2]

# 혹시 모를 결측치 제거
train_mecab = train_mecab.dropna(axis=1)
test_mecab = test_mecab.dropna(axis=1)

# 컬럼명 변경
train_mecab.columns = ['문장', 'clean', 'violence']
test_mecab.columns = ['문장', 'clean', 'violence']

In [32]:
# 분할
train_kiwi = train_df[['kiwi','clean','violence']]
test_kiwi = test_df[['kiwi','clean','violence']]

# 각각 길이가 2미만 제거 
train_kiwi = train_kiwi[train_kiwi['kiwi'].str.len() >= 2]
test_kiwi = test_kiwi[test_kiwi['kiwi'].str.len() >= 2]

# 혹시 모를 결측치 제거
train_kiwi = train_kiwi.dropna(axis=1)
test_kiwi = test_kiwi.dropna(axis=1)

# 컬럼명 변경
train_kiwi.columns = ['문장', 'clean', 'violence']
test_kiwi.columns = ['문장', 'clean', 'violence']

In [33]:
# 분할
train_khaiii = train_df[['khaiii','clean','violence']]
test_khaiii = test_df[['khaiii','clean','violence']]

# 각각 길이가 2미만 제거 
train_khaiii = train_khaiii[train_khaiii['khaiii'].str.len() >= 2]
test_khaiii = test_khaiii[test_khaiii['khaiii'].str.len() >= 2]

# 혹시 모를 결측치 제거
train_khaiii = train_khaiii.dropna(axis=1)
test_khaiii = test_khaiii.dropna(axis=1)

# 컬럼명 변경
train_khaiii.columns = ['문장', 'clean', 'violence']
test_khaiii.columns = ['문장', 'clean', 'violence']

# 06.파일 저장

In [34]:
train_mecab.to_csv(path + "violence_train_mecab.csv", index=False, encoding="utf-8-sig")
test_mecab.to_csv(path + "violence_test_mecab.csv", index=False, encoding="utf-8-sig")

In [36]:
train_kiwi.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/train_kiwi.csv", index=False, encoding="utf-8-sig")
test_kiwi.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/test_kiwi.csv", index=False, encoding="utf-8-sig")

In [ ]:
train_khaiii.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/train_khaiii.csv", index=False, encoding="utf-8-sig")
test_khaiii.to_csv("/content/drive/MyDrive/NLP미니플젝-10조/태훈/코드/형태소분석기/data3/test_khaiii.csv", index=False, encoding="utf-8-sig")